## Exercício LangChain e Gemini - ALURA

- Exercício proposto pro curso de langChain e python - ALURA - Cração dde cadeia simples: 
- Emoções (Roda das Emoções de Plutchik)
- Distorções cognitivas
- Cartões de enfrentamento

- No exercício envolvi assuntos de interesse pessoal e tive por objetivo usar o conceito de Simple Sequencial Chain para analisar o conteúdo emocional e distorções cognitivas de um trecho da crônica "Se eu fosse eu" de Clarice Lispector e a partir da análise propor cartões de enfrentamento para pensamentos disfuncionais.   

### Imports

In [8]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

### Carregamento da Chave API -> Utilização do Gemini 

In [2]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

### Instancia da LLM

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    google_api_key=api_key
)

### Definição dos templates de prompts

In [4]:
modelo_emocoes = ChatPromptTemplate.from_template(
    "Identifique as emoções presentes na seguinte texto conforme a teoria da roda das emoções. texto: {texto}"
)

modelo_distorcoes_cognitivas = ChatPromptTemplate.from_template(
    "Identifique as distorções cognitivas a seguinte analise emocional: {emoções}"
)

modelo_pensamentos_automaticos = ChatPromptTemplate.from_template(
    "Sugira pensamentos saudáveis de enfrentamento dadas as seguintes distorções cognitivas: {distorções}. Vc deve montar um esquema tipo cartões de enfrentamento, estilo distorção -> pensamento saudável"
)


### Criação das Cadeias

In [5]:
cadeia_emocoes = LLMChain(
    llm=llm,
    prompt=modelo_emocoes
)
cadeia_distorcoes = LLMChain(
    llm=llm,
    prompt=modelo_distorcoes_cognitivas
)

cadeia_pensamentos = LLMChain(
    llm=llm,
    prompt=modelo_pensamentos_automaticos
)

# criação de uma sequência de cadeias -  simple sequential chain
# usa a saída de uma cadeia como entrada da próxima
# a saída da cadeia_emocoes é a entrada da cadeia_distorcoes, e assim por diante

cadeia = SimpleSequentialChain(
    chains=[cadeia_emocoes, cadeia_distorcoes, cadeia_pensamentos],
    verbose=True # pra ter detalhes da execução passo a passo no terminal
)


C:\Users\syann\AppData\Local\Temp\ipykernel_17856\1255668256.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  cadeia_emocoes = LLMChain(


### Execução da Cadeia 

In [10]:
texto = """E não me sinto bem. Experimente: se você fosse você, como seria e o que faria? 
Logo de início se sente um constrangimento: a mentira em que nos acomodamos acabou de ser 
levemente locomovida do lugar onde se acomodara. No entanto já li biografias de pessoas que 
de repente passavam a ser elas mesmas, e mudavam inteiramente de vida. Acho que se eu fosse 
realmente eu, os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado. Como? Não sei."""

resultado = cadeia.invoke(texto)
print(resultado)




> Entering new SimpleSequentialChain chain...
O texto expressa principalmente **constrangimento** e **medo**,  ambos presentes na Roda das Emoções de Plutchik. Vamos analisar:

* **Constrangimento:**  A frase "E não me sinto bem" e a descrição da "mentira em que nos acomodamos" apontam diretamente para uma sensação de desconforto e vergonha, características do constrangimento. A mentira provavelmente envolve uma incoerência entre a imagem que a pessoa projeta e sua verdadeira identidade.

* **Medo:** A  ideia de mudar completamente de vida e a consequência imaginada de não ser mais reconhecido pelos amigos ("os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado") revelam um medo da mudança, do desconhecido e das potenciais consequências negativas de ser autêntico.  A incerteza ("Como? Não sei") reforça esse medo.

Embora não seja explicitamente mencionado, podemos inferir a presença de **tristeza** subjacente ao constrangimento e medo. A insatisfação expres

### Utilização de Parser

Proposta:
- Analisar emoções e retornar uma estrutura com a lista de emoções e a justificativa das emoções
- analisar sentimentos relacionados com as emoções detectadas anteriormente


In [11]:
from langchain_core.pydantic_v1 import Field, BaseModel
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

## criação da classe pro parser
class AnaliseEmocional(BaseModel):
    emoções: list[str] = Field(description="Emoções identificadas no texto")
    justificativa: str = Field(description="Justificativa para as emoções identificadas")

## definição do parser
parser = JsonOutputParser(pydantic_object=AnaliseEmocional)

## modelos prompts com o parser
modelo_emocoes = PromptTemplate(
    template ="""Identifique as emoções presentes na seguinte texto conforme a teoria da roda das emoções.
    Texto: {texto}
    {formatacao_de_saida}
    """,
    input_variables=["texto"],
    partial_variables={"formatacao_de_saida": parser.get_format_instructions()}
) 


classificacao_sentimentos = ChatPromptTemplate.from_template(
    "Classifique a potcentagem dos sentimentos(positivo, negativo ou neutro) presentas na lista de emoções. Ex: 'Na lista estão presentes 50% de emoções negativas e 50% de positivas.' Emoções: {emoções}"
)

## criação das cadeia, uma a uma 

cadeia_emocoes = LLMChain(
    llm=llm,
    prompt=modelo_emocoes
)

cadeia_classificacao_sentimentos = LLMChain(
    llm=llm,
    prompt=classificacao_sentimentos
)

cadeia = SimpleSequentialChain(
    chains= [cadeia_emocoes, cadeia_classificacao_sentimentos],
    verbose=True
)


# criação de uma sequência de cadeias -  simple sequential chain
# usa a saída de uma cadeia como entrada da próxima
# a saída da cadeia_emocoes é a entrada da cadeia_distorcoes, e assim por diante

cadeia = SimpleSequentialChain(
    chains= [cadeia_emocoes, cadeia_classificacao_sentimentos],
    verbose=True
)

# execução da cadeia
texto = """E não me sinto bem. Experimente: se você fosse você, como seria e o que faria? 
Logo de início se sente um constrangimento: a mentira em que nos acomodamos acabou de ser 
levemente locomovida do lugar onde se acomodara. No entanto já li biografias de pessoas que 
de repente passavam a ser elas mesmas, e mudavam inteiramente de vida. Acho que se eu fosse 
realmente eu, os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado. Como? Não sei."""

resultado = cadeia.invoke(texto)



> Entering new SimpleSequentialChain chain...
```json
{
  "emoções": ["constrangimento", "medo", "insegurança", "desconforto"],
  "justificativa": "O texto expressa um desconforto com a própria situação, revelando um sentimento de constrangimento por uma mentira mantida. A frase \"E não me sinto bem\" indica desconforto. A dúvida sobre como seria se fosse realmente si mesmo, e a consequente especulação de que seus amigos não o reconheceriam (\"os amigos não me cumprimentariam na rua porque até minha fisionomia teria mudado\"), revela medo e insegurança quanto às mudanças e suas consequências. A incerteza expressa em \"Como? Não sei\" reforça a insegurança e o desconforto da situação."
}
```
Na lista estão presentes 100% de emoções negativas.  Não há emoções positivas ou neutras presentes.

> Finished chain.
